<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/NaiveBayes_Spam_Classifier/blob/master/NaiveBayes_Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NaiveBayes Spam Classifier**

## **Libraries**

In [1]:
import pandas as pd
import string
import math
from collections import defaultdict